# Urea with GPAW

This is an xHARPy example file for demonstrating how a Hirshfeld Atom Refinement with GPAW is set up.

In [1]:
from xharpy import (shelxl_hkl2pd, cif2data, lst2constraint_dict,
                    create_construction_instructions, refine, write_cif,
                    write_res, write_fcf, ConstrainedValues,
                    create_atom_table)
import os

In [2]:
from xharpy.core import construct_esds

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
output_folder = 'xharpy_output'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    
filename = 'xharpy'

In [5]:
atom_table, cell, cell_esd, symm_mats_vecs, symm_strings, wavelength  = cif2data('iam.cif', 0)

In [6]:
atom_table['adp_type'] = 'Uani'
atom_table[['U_11', 'U_22', 'U_33']] = atom_table[['U_11', 'U_22', 'U_33']].fillna(0.01)
atom_table[['U_12', 'U_23', 'U_13']] = atom_table[['U_12', 'U_23', 'U_13']].fillna(0.00)

In [7]:
hkl = shelxl_hkl2pd('iam.hkl')

In [8]:
constraint_dict = lst2constraint_dict(
    'iam.lst'
)

In [9]:
refinement_dict = {
    'f0j_source': 'gpaw_mpi',
    'reload_step': 1,
    'core': 'constant',
}

In [10]:
construction_instructions, parameters = create_construction_instructions(
    atom_table=atom_table,
    constraint_dict=constraint_dict,
    refinement_dict=refinement_dict
)


In [ ]:
refinement_dict['core_io'] = ('save', os.path.join(output_folder, 'f0j_core.pic'))

computation_dict = {
    'xc': 'SCAN',
    'txt': os.path.join(output_folder, 'gpaw.txt'),
    'mode': 'fd',
    'h': 0.225,
    'gridinterpolation': 4,
    #'poissonsolver': poissonsolver,d
    'average_symmequiv': False,
    'convergence':{'density': 1e-7},
    #'kpts': {'size': (2, 2, 2), 'gamma': True},
    'kpts': {'size': (1, 1, 1), 'gamma': True},
    'symmetry': {'symmorphic': False},
    'nbands': -2,
    'save_file': os.path.join(output_folder, 'gpaw_result.gpw')
}
parameters, var_cov_mat, information = refine(
    cell=cell, 
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict
)

Started refinement at  2022-01-04 13:36:50.314545
Preparing


In [ ]:
refinement_dict['core_io'] = ('load', os.path.join(output_folder, 'f0j_core.pic'))

computation_dict = {
    'xc': 'SCAN',
    'txt': os.path.join(output_folder, 'gpaw.txt'),
    'mode': 'fd',
    'h': 0.125,
    'gridinterpolation': 4,
    #'poissonsolver': poissonsolver,d
    'average_symmequiv': False,
    'convergence':{'density': 1e-7},
    'kpts': {'size': (2, 2, 2), 'gamma': True},
    #'kpts': {'size': (1, 1, 1), 'gamma': True},
    'symmetry': {'symmorphic': False},
    'nbands': -2
    'save_file': os.path.join(output_folder, 'gpaw_result.gpw')
}
parameters, var_cov_mat, information = refine(
    cell=cell, 
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict
)

In [ ]:
write_fcf(
    fcf_path=os.path.join(output_folder, filename + '.fcf'),
    fcf_dataset='xharpy',
    fcf_mode=4,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [ ]:
write_fcf(
    fcf_path=os.path.join(output_folder, filename + '_6.fcf'),
    fcf_dataset='xharpy_6',
    fcf_mode=6,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [ ]:
write_res(
    out_res_path=os.path.join(output_folder, filename + '_6.res'),
    in_res_path='iam.lst',
    cell=cell,
    cell_esd=cell_esd,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength
)

In [ ]:
write_cif(
    output_cif_path=os.path.join(output_folder, filename + '.cif'),
    cif_dataset=filename,
    shelx_cif_path='iam.cif',
    shelx_dataset=0,
    cell=cell,
    cell_esd=cell_esd,
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    var_cov_mat=var_cov_mat,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict,
    information=information
)

In [ ]:
atom_table = create_atom_table(cell, construction_instructions, parameters, var_cov_mat)

In [ ]:
constraint_dict['O']['cijk'] = ConstrainedValues(
    variable_indexes = (-1, -1, 0, -1, -1, 1, -1, 1, -1, 2),
    multiplicators = (0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0),
    added_value = tuple([0]* 10),
    special_position=True
)
constraint_dict['C']['cijk'] = ConstrainedValues(
    variable_indexes = (-1, -1, 0, -1, -1, 1, -1, 1, -1, 2),
    multiplicators = (0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0),
    added_value = tuple([0.0]* 10),
    special_position=True
)
constraint_dict['N']['cijk'] = ConstrainedValues(
    variable_indexes=(0, 0, 1, 2, 2, 3, 4, 3, 4, 5),
    multiplicators=tuple([1.0]*10),
    added_value=tuple([0.0]*10),
    special_position=True
)


construction_instructions, parameters = create_construction_instructions(
    atom_table=atom_table,
    constraint_dict=constraint_dict,
    refinement_dict=refinement_dict,
    atoms_for_gc3=['C', 'N', 'O'],
    scaling0=float(parameters[0])
)

In [ ]:
refinement_dict['reload_step'] = 0

In [ ]:
parameters, var_cov_mat, information = refine(
    cell=cell, 
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict
)